In [1]:
!pip install kagglehub pandas scikit-learn statsmodels "numpy<2"

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/40.1 kB ? eta -:--:--
     ---------- ----------------------------- 10.2/40.1 kB ? eta -:--:--
     ------------------- ------------------ 20.5/40.1 kB 217.9 kB/s eta 0:00:01
     -------------------------------------- 40.1/40.1 kB 271.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/70.6 kB ? eta -:--:--
   ---------------------------------------- 70.6/70.6 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/160.4 kB ? eta -:--:--
   -------------------------------------- - 153.6/160.4 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 160.4/160.4 kB 2.4 MB/s eta 0:00:00


In [1]:
import os
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import average_precision_score, confusion_matrix, classification_report

print("Baixando dataset...")
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
csv_file = [f for f in os.listdir(path) if f.endswith('.csv')][0]
df = pd.read_csv(os.path.join(path, csv_file))

print(f"Base carregada! Temos {df.shape[0]} transações.")

Baixando dataset...


100%|██████████| 66.0M/66.0M [00:22<00:00, 3.11MB/s]

Extracting files...


Base carregada! Temos 284807 transações.


In [2]:
# Padronizando Time e Amount
scaler = StandardScaler()
df['Time_Scaled'] = scaler.fit_transform(df['Time'].values.reshape(-1, 1))
df['Amount_Scaled'] = scaler.fit_transform(df['Amount'].values.reshape(-1, 1))

# Removendo as colunas antigas e definindo X e y
df = df.drop(columns=['Time', 'Amount'])

# O Alvo é a coluna Class (0 ou 1)
y = df['Class']
X = df.drop(columns=['Class'])

print("Dados padronizados e prontos para o modelo!")

Dados padronizados e prontos para o modelo!


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import average_precision_score, confusion_matrix
import numpy as np

# Separação em Treino e Teste (Estratificado para manter a proporção de fraudes)
# Usaremos o mesmo random_state para a comparação ser justa
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Treinando o modelo de Regressão Linear Clássica...")
modelo_linear = LinearRegression()
modelo_linear.fit(X_train, y_train)

# 1. O modelo gera as previsões contínuas (ex: -0.1, 0.4, 0.9)
y_pred_continuo = modelo_linear.predict(X_test)

# 2. O "Hack" do Limiar: Tudo >= 0.5 vira 1 (Fraude). O resto vira 0.
limiar = 0.5
y_pred_binario = (y_pred_continuo >= limiar).astype(int)

# Avaliação usando AUPRC (Usamos o valor contínuo como se fosse o "grau de certeza")
auprc = average_precision_score(y_test, y_pred_continuo)

print("\n" + "="*40)
print(" RESULTADOS: REGRESSÃO LINEAR")
print("="*40)
print(f"AUPRC Score: {auprc:.4f}")

print("\nMatriz de Confusão (Usando Limiar de 0.5):")
cm = confusion_matrix(y_test, y_pred_binario)
print(f"Transações Legítimas identificadas corretamente: {cm[0][0]}")
print(f"Transações Legítimas bloqueadas por engano (Falso Positivo): {cm[0][1]}")
print(f"FRAUDES que passaram direto (Falso Negativo): {cm[1][0]}  <-- O PESADELO DO BANCO")
print(f"FRAUDES bloqueadas corretamente: {cm[1][1]}")

Treinando o modelo de Regressão Linear Clássica...

 RESULTADOS: REGRESSÃO LINEAR
AUPRC Score: 0.7196

Matriz de Confusão (Usando Limiar de 0.5):
Transações Legítimas identificadas corretamente: 56851
Transações Legítimas bloqueadas por engano (Falso Positivo): 13
FRAUDES que passaram direto (Falso Negativo): 53  <-- O PESADELO DO BANCO
FRAUDES bloqueadas corretamente: 45


In [4]:
import numpy as np
from sklearn.metrics import confusion_matrix

print("TESTANDO DIFERENTES LIMIARES (THRESHOLDS) NA REGRESSÃO LINEAR")
print("="*60)

# Vamos testar limites desde 0.05 até 0.5
limiares_para_testar = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

for limiar in limiares_para_testar:
    # Transforma em 1 (Fraude) tudo que for maior que o limiar atual
    y_pred_simulado = (y_pred_continuo >= limiar).astype(int)
    cm = confusion_matrix(y_test, y_pred_simulado)
    
    fraudes_pegas = cm[1][1]
    fraudes_perdidas = cm[1][0]
    cartoes_bloqueados_engano = cm[0][1]
    
    print(f"--- Limiar de Corte: {limiar} ---")
    print(f"FRAUDES PEGAs com sucesso: {fraudes_pegas}")
    print(f"Fraudes perdidas (Prejuízo): {fraudes_perdidas}")
    print(f"Cartões bloqueados por engano (Irritação do cliente): {cartoes_bloqueados_engano}")
    print("-" * 30)

TESTANDO DIFERENTES LIMIARES (THRESHOLDS) NA REGRESSÃO LINEAR
--- Limiar de Corte: 0.05 ---
FRAUDES PEGAs com sucesso: 80
Fraudes perdidas (Prejuízo): 18
Cartões bloqueados por engano (Irritação do cliente): 23
------------------------------
--- Limiar de Corte: 0.1 ---
FRAUDES PEGAs com sucesso: 80
Fraudes perdidas (Prejuízo): 18
Cartões bloqueados por engano (Irritação do cliente): 23
------------------------------
--- Limiar de Corte: 0.2 ---
FRAUDES PEGAs com sucesso: 80
Fraudes perdidas (Prejuízo): 18
Cartões bloqueados por engano (Irritação do cliente): 20
------------------------------
--- Limiar de Corte: 0.3 ---
FRAUDES PEGAs com sucesso: 72
Fraudes perdidas (Prejuízo): 26
Cartões bloqueados por engano (Irritação do cliente): 17
------------------------------
--- Limiar de Corte: 0.4 ---
FRAUDES PEGAs com sucesso: 56
Fraudes perdidas (Prejuízo): 42
Cartões bloqueados por engano (Irritação do cliente): 13
------------------------------
--- Limiar de Corte: 0.5 ---
FRAUDES PEGAs

In [5]:
import pandas as pd
from sklearn.utils import resample

print("APLICANDO UNDERSAMPLING PARA BALANCEAR OS DADOS...")

# 1. Juntamos o X_train e y_train para facilitar a manipulação
df_train = pd.concat([X_train, y_train], axis=1)

# 2. Separamos as fraudes e as transações legítimas
fraudes = df_train[df_train['Class'] == 1]
legitimas = df_train[df_train['Class'] == 0]

# 3. Sorteamos aleatoriamente transações legítimas na mesma quantidade de fraudes
legitimas_undersampled = resample(
    legitimas, 
    replace=False,           # Sem repetição
    n_samples=len(fraudes),  # Exatamente o mesmo número de fraudes
    random_state=42
)

# 4. Juntamos tudo em um novo dataset de treino 50/50
df_train_balanceado = pd.concat([fraudes, legitimas_undersampled])

# Separamos de volta em X e y para o modelo
X_train_bal = df_train_balanceado.drop('Class', axis=1)
y_train_bal = df_train_balanceado['Class']

print(f"Novo tamanho do treino: {len(X_train_bal)} linhas (50% Fraude / 50% Legítimo).")

# 5. Treinando a Nova Regressão Linear
modelo_balanceado = LinearRegression()
modelo_balanceado.fit(X_train_bal, y_train_bal)

# Previsões nos dados de TESTE originais (que continuam desbalanceados, a vida real!)
y_pred_cont_bal = modelo_balanceado.predict(X_test)

# Como a base agora é 50/50, o limiar natural de 0.5 volta a fazer sentido matemático
y_pred_bin_bal = (y_pred_cont_bal >= 0.5).astype(int)

print("\n" + "="*40)
print(" RESULTADOS: REGRESSÃO LINEAR COM UNDERSAMPLING")
print("="*40)

cm_bal = confusion_matrix(y_test, y_pred_bin_bal)
print(f"FRAUDES PEGAs com sucesso: {cm_bal[1][1]}")
print(f"Fraudes perdidas (Prejuízo): {cm_bal[1][0]}")
print(f"Cartões bloqueados por engano (Irritação do cliente): {cm_bal[0][1]}")
print(f"Transações Legítimas identificadas corretamente: {cm_bal[0][0]}")

APLICANDO UNDERSAMPLING PARA BALANCEAR OS DADOS...
Novo tamanho do treino: 788 linhas (50% Fraude / 50% Legítimo).

 RESULTADOS: REGRESSÃO LINEAR COM UNDERSAMPLING
FRAUDES PEGAs com sucesso: 82
Fraudes perdidas (Prejuízo): 16
Cartões bloqueados por engano (Irritação do cliente): 900
Transações Legítimas identificadas corretamente: 55964


In [6]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
import pandas as pd

print("APLICANDO SMOTE (OVERSAMPLING INTELIGENTE)...")

# Inicializa o SMOTE
smote = SMOTE(random_state=42)

# A regra de ouro: aplicamos o balanceamento APENAS nos dados de TREINO.
# O teste (y_test) tem que continuar sendo a vida real (desbalanceado).
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print(f"Novo tamanho do treino: {len(X_train_smote)} linhas.")
print(f"Fraudes: {sum(y_train_smote == 1)} | Legítimas: {sum(y_train_smote == 0)}")

# Treinando a Regressão Linear na base gigante e balanceada
modelo_smote = LinearRegression()
modelo_smote.fit(X_train_smote, y_train_smote)

# Previsão nos dados de Teste intocados
y_pred_cont_smote = modelo_smote.predict(X_test)

# Limiar de 0.5 (como a base de treino era 50/50, o meio é o ponto de corte ideal)
y_pred_bin_smote = (y_pred_cont_smote >= 0.5).astype(int)

print("\n" + "="*40)
print(" RESULTADOS: REGRESSÃO LINEAR COM SMOTE")
print("="*40)

cm_smote = confusion_matrix(y_test, y_pred_bin_smote)
print(f"FRAUDES PEGAs com sucesso: {cm_smote[1][1]}")
print(f"Fraudes perdidas (Prejuízo): {cm_smote[1][0]}")
print(f"Cartões bloqueados por engano (Irritação do cliente): {cm_smote[0][1]}")
print(f"Transações Legítimas identificadas corretamente: {cm_smote[0][0]}")

APLICANDO SMOTE (OVERSAMPLING INTELIGENTE)...
Novo tamanho do treino: 454902 linhas.
Fraudes: 227451 | Legítimas: 227451

 RESULTADOS: REGRESSÃO LINEAR COM SMOTE
FRAUDES PEGAs com sucesso: 85
Fraudes perdidas (Prejuízo): 13
Cartões bloqueados por engano (Irritação do cliente): 794
Transações Legítimas identificadas corretamente: 56070


In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

print("Calculando o VIF para provar o efeito do PCA...")

# Pegamos apenas as colunas de V1 a V28
colunas_pca = [f'V{i}' for i in range(1, 29)]
X_pca = X_train[colunas_pca]

vif_data = pd.DataFrame()
vif_data["Variável"] = X_pca.columns

vif_data["VIF"] = [variance_inflation_factor(X_pca.values, i) for i in range(X_pca.shape[1])]

print("\n--- RESULTADO DO TESTE VIF (A Mágica do PCA) ---")
print(vif_data.head(10).to_string(index=False)) # Mostrando as 10 primeiras para não poluir a tela
print("...")

Calculando o VIF para provar o efeito do PCA...

--- RESULTADO DO TESTE VIF (A Mágica do PCA) ---
Variável      VIF
      V1 1.000126
      V2 1.000098
      V3 1.000167
      V4 1.000070
      V5 1.000220
      V6 1.000055
      V7 1.000238
      V8 1.000102
      V9 1.000058
     V10 1.000094
...
